## Data Assimilation Workflow

1. Get ignition, ignite_timestamp
2. Get observation, observe_timestamp
3. Choose weather station and obtain windspeed/winddirection
4. Get model: Use ignition and wind at ignition timestamp to run a simulation.
5. Get nextobservation, nextobserve_timestamp
6. Get modifiedignition: Feed ignition, observation, weather, model and nextobservation into Kalman Filter
7. Get nextmodel: Use modifiedmodel as the next ignition

In [1]:
from shapely.geometry import MultiPolygon, Polygon

import pandas as pd
import geopandas as gpd
import numpy as np

import datetime

from matplotlib import pyplot as plt

import requests
import arrow
import uuid

import sys
sys.path.append('/home/jovyan/python-helper/src/')

from loggers import TimeEstimator
from branca.colormap import LinearColormap

from matplotlib.colors import LinearSegmentedColormap

import farsiteutils_v2 as futils
from kalmanutils import interpolate_perimeter, get_vertices, calculate_vectors, align_perimeters, calculate_uncertainties_observed, calculate_modified, calculate_parameters

%load_ext autoreload
%autoreload 2

In [2]:
def plot_perimeter(vertices, ax, **kwargs):
    ax.plot(np.array(vertices)[:,0], 
            np.array(vertices)[:,1], **kwargs)

In [3]:
fp = futils.FilePaths('/home/jovyan/data/')
usr = futils.User(fp)

usr.db.gdfignitionAll = usr.db.gdfignitionAll.to_crs(epsg=5070)

Database interaction not yet implemented. Use pickle file for dataframes instead!


### Choose ignition

In [4]:
igniteidx = 'f23196b034474744bdca7df94b13e0f7'
compareidx = '9f82e870591748a9a8a01346d174f2a1'
lcpidx = '43b7f5db36994599861eec4849cc68fd'
barrieridx = 'cb47616cd2dc4ccc8fd523bd3a5064bb'

ignite_geom = usr.db.gdfignitionAll.loc[igniteidx, 'geometry']

ignition = get_vertices(ignite_geom)

### Choose input parameters for simulation

In [5]:
wind_speed = 16
wind_direction = 90

inputData = {'description': 'Maria2019',
             'igniteidx'  : igniteidx,
             'compareidx' : compareidx,
             'lcpidx'     : lcpidx,
             'barrieridx' : barrieridx,

             'windspeed': wind_speed, 'winddirection': wind_direction,
             'relhumid': 90, 'temperature': 20}

# Setup the mainapi for this calculation
mainapi = usr.calculatePerimeters(inputData)

{'description': 'Maria2019', 'igniteidx': 'f23196b034474744bdca7df94b13e0f7', 'compareidx': '9f82e870591748a9a8a01346d174f2a1', 'lcpidx': '43b7f5db36994599861eec4849cc68fd', 'barrieridx': 'cb47616cd2dc4ccc8fd523bd3a5064bb', 'windspeed': 16, 'winddirection': 90, 'relhumid': 90, 'temperature': 20}
Choosing a perimeter from the database


In [ ]:
mainapi.run_farsite()

### Obtain model output

In [ ]:
model_geom = usr.db.gdfsimulation.iloc[-1]['geometry'].geoms[0]
model = get_vertices(model_geom)

### Collect observation

In [ ]:
observe_geom = usr.db.gdfignitionAll.loc[compareidx, 'geometry']
observe = get_vertices(observe_geom)

In [ ]:
# Calculate dt from observation and ignition
dt = int((usr.db.gdfignitionAll.loc[compareidx, 'datetime'] - usr.db.gdfignitionAll.loc[igniteidx, 'datetime']).total_seconds()/60)

### Rearrange the parameters to align, calculate velocities

In [ ]:
ignition_aligned, model_velocity_aligned, model_aligned, observe_aligned, observed_velocity_aligned, ignition_uncertainties = calculate_parameters(ignition, observe, model, wind_direction, dt)

### Calculate Kalman parameters

In [ ]:
# State transition matrix
F = np.eye(4*len(ignition_aligned))
for i in range(len(F)):
    if i%2 == 0:
        F[i,i+1] = dt

# State Vector
X = np.zeros((4*len(ignition_aligned), 1))
# Observation
Z = np.zeros((4*len(observe_aligned), 1))
Xpre = np.zeros((4*len(ignition_aligned), 1))
# Observed uncertainty
R = np.zeros_like(F)
# Covariance matrix
P = np.zeros_like(F)
# Measurement matrix
H = np.eye(len(X))
for i in range(len(ignition_aligned)):
    X[4*i] = ignition_aligned[i,0]
    X[4*i + 1] = model_velocity_aligned[i,0]
    X[4*i + 2] = ignition_aligned[i,1]
    X[4*i + 3] = model_velocity_aligned[i,1]
    
    Xpre[4*i] = model_aligned[i,0]
    Xpre[4*i + 1] = model_velocity_aligned[i,0]
    Xpre[4*i + 2] = model_aligned[i,1]
    Xpre[4*i + 3] = model_velocity_aligned[i,1]
    
    Z[4*i] = observe_aligned[i,0]
    Z[4*i + 1] = observed_velocity_aligned[i,0]
    Z[4*i + 2] = observe_aligned[i,1]
    Z[4*i + 3] = observed_velocity_aligned[i,1]
        
    R[[4*i, 4*i+1, 4*i+2, 4*i+3], [4*i, 4*i+1, 4*i+2, 4*i+3]] = ignition_uncertainties[i]
    P[[4*i, 4*i+1, 4*i+2, 4*i+3], [4*i, 4*i+1, 4*i+2, 4*i+3]] = 0.1

In [ ]:
kf = KalmanFilter(len(X), len(Z))
kf.F = F
kf.R = R
kf.Q = 0
kf.P = P
kf.x = X
kf.H = H

In [ ]:
%%time
kf.update(Z)

In [ ]:
Xmodified = kf.x.copy()

In [ ]:
xmod, vxmod, ymod, vymod = Xmodified[::4], Xmodified[1::4], Xmodified[2::4], Xmodified[3::4]
modified_vertices = list(zip(xmod, ymod))
modified_geom = Polygon(shell=modified_vertices)
gpd.GeoDataFrame(data={'FID': 0}, geometry = [modified_geom.buffer(0)], index=[0], crs='epsg:5070').to_file('/home/jovyan/farsite/inputs/maria_ignite/maria_21231_mod.shp')

### Add the modified geom to the ignition database

In [ ]:
usr.db.gdfignitionAll = usr.db.gdfignitionAll.append(
    gpd.GeoDataFrame(data={'filetype': 'Ignition',
                       'objectid': None,
                       'filepath': '/home/jovyan/farsite/inputs/maria_ignite/maria_21231_mod.shp',
                       'datetime': usr.db.gdfignitionAll.loc[igniteidx, 'datetime'],
                       'description': 'Maria2019'},
                 geometry=[modified_geom], index=[uuid.uuid4().hex])
)                      
                       

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(4,2), dpi=400)

plot_perimeter(ignition_aligned, ax=ax, color='blue')
# plot_perimeter(modified_vertices, ax=ax, ls='--')
plot_perimeter(observe_aligned, ax=ax, color='tab:orange')
# # plot_perimeter(next_observe_aligned, ax=ax, color='tab:red')
plot_perimeter(model_aligned, color='red', ax=ax)
ax.plot(np.mean(ignition_aligned, axis=0)[0], np.mean(ignition_aligned, axis=0)[1], 'o', color='blue')

ax.set_aspect(1)
    
xlim = (-2086157.8176353823, -2081650.1039221247)
ylim = (1502205.458511021, 1505619.1686423323)

ax.set_xlim(xlim)
ax.set_ylim(ylim)

ax.set_xlabel('East-West (m)')
ax.set_ylabel('North-South (m)')